In [ ]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
print(len(channels))
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(1. * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)

core_list = [hex(2**n) for n in range(20)]

card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

In [ ]:
max_f = 1.e6
n_tweezers = 2

# f_list = [75.e6 - max_f,75.e6,75.e6 + max_f]
# f_list = [75.e6 - max_f,75.e6 + max_f]
f_list = np.linspace(75.e6 - max_f,75.e6 + max_f,n_tweezers)
# f_list = [75.e6]
# a_list = np.linspace(1/n_tweezers,1/n_tweezers,n_tweezers)
a_list = [.1,.11]

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)
dds.exec_at_trg()
dds.write_to_card()

In [ ]:
carrier_freq  =   75.e6
modulation_freq  = 1.e6
modulation_depth = 2.e6
period_s = 20.e-3
num_samples = int(1/(period_s * modulation_freq))

sample_range = np.arange(num_samples)*period_s

freq_lists = np.empty([len(f_list),len(sample_range)])

for i in len(f_list):
    freq_lists[i] = f_list[i] + modulation_depth * np.sin(2*np.pi*sample_range*modulation_freq)

# STEP 1 - Pre-fill Buffer
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(period_s)
dds.exec_at_trg()
fill_max = dds.queue_cmd_max()
counter = 0
for counter in range(fill_max // 2 - 4):
    for tone_idx in np.shape(freq_lists)[0]:
        freq_Hz = freq_lists[tone_idx][counter % num_samples]
        dds[tone_idx].freq(freq_Hz)
    dds.exec_at_trg()
dds.write_to_card()

fill_count = dds.queue_cmd_count()
print("Pre-fill buffer: {}/{}".format(fill_count, fill_max))

# STEP 2 - Streaming data points
fill_number = fill_max // 4
fill_check = fill_max - fill_number
while True: # infinitely long streaming
    while True:
        fill_count = dds.queue_cmd_count()
        if fill_count < fill_check: break
    print("Adding a block of commands to buffer")
    for i in range(fill_number // 2):
        for tone_idx in np.shape(freq_lists)[0]:
            freq_Hz = freq_lists[tone_idx][counter % num_samples]
            dds[tone_idx].freq(freq_Hz)
            dds.exec_at_trg()
            counter += 1
            counter %= num_samples
    dds.write_to_card()
    status = dds.status()
    if status & spcm.SPCM_DDS_STAT_QUEUE_UNDERRUN:
        break

In [ ]:
# card.close(card._handle)
card.stop()

card.close(card._handle)